# ANÁLISIS DE REDES NEURONALES PREENTRENADAS PARA LA ESTIMACIÓN DE PROFUNDIDAD

Se pretende realizar el análisis de una serie de redes neuronales preentrenadas para la estimación de profundidad que son: 
- MonoDepth2
- MiDAS
- SC-DepthV2
- DenseDepth

Para poder comprobar su precisión se han sacado unas imágenes de test con una cámara Interl RealSense tanto en modo monocular como en RGBD que se han guardado en sus respectivas carpetas ./monocular_photos y ./rgbd_photos. Las imágenes monoculares servirán como entrada para las redes y las rgbd se emplearán como ground truth y así obtener la precisión de los mapas de profundidad de la salida de las redes.

A mayores también se pueden hacer pruebas con los datasets de repositorios públicos como: 

- https://theairlab.org/tartanair-dataset/
- https://cvg.cit.tum.de/data/datasets/rgbd-dataset/download
- https://projects.asl.ethz.ch/datasets/doku.php?id=kmavvisualinertialdatasets
- https://www.cvlibs.net/datasets/kitti/

Sin embargo, el problema es que en este caso puede que alguna de las redes haya sido entrenada con las imágenes de alguno o varios de esos repositorios, y por lo tanto el test no será fiable. Deberíamos de estar seguros que las imágenes nunca hayan sido vistas por las redes. 


## MonoDepth2

https://github.com/nianticlabs/monodepth2

En concreto el modelo mono_640x192 está entrenado solo con imágenes monoculares, sin información estéreo. <br>
Por otro lado, el modelo mono+stereo_640x192 está entrenado usando tanto imágenes monoculares como pares estéreo durante el entrenamiento. <br>
Aunque solo utiliza monoculares para la predicción, el entrenamiento con datos estéreo puede ayudar a mejorar la precisión. <br><br>

Por lo tanto, el mejor modelo y el escogido para mi caso es el mono+stereo_640x192.

1. Descarga del modelo preentrenado.

In [3]:
!python3 ./test_simple.py --image_path monocular_photos --model_name mono+stereo_640x192 --ext jpg

-> Downloading pretrained model to models/mono+stereo_640x192.zip
   Unzipping model...
   Model unzipped to models/mono+stereo_640x192
-> Loading model from  models/mono+stereo_640x192
   Loading pretrained encoder
/home/andrea/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/andrea/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
   Loading pretrained decoder
Traceback (most recent call last):
  File "./test_simple.py", line 171, in <module>
    test_simple(args)
  File "./test_simple.py", line 112, in test_simple
    raise Exception("Can not find args.image

In [1]:
import os
import torch
from torchvision import transforms, datasets
from PIL import Image

In [4]:
# Configuración de la carpeta de imágenes y resultados
input_folder = './monocular_photos'
output_folder = './network_depth_maps'
model_name = 'mono+stereo_640x192'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


In [6]:
def download_model_if_doesnt_exist(model_name):
    """If pretrained kitti model doesn't exist, download and unzip it
    """
    # values are tuples of (<google cloud URL>, <md5 checksum>)
    download_paths = {
        "mono_640x192":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/mono_640x192.zip",
             "a964b8356e08a02d009609d9e3928f7c"),
        "stereo_640x192":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/stereo_640x192.zip",
             "3dfb76bcff0786e4ec07ac00f658dd07"),
        "mono+stereo_640x192":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/mono%2Bstereo_640x192.zip",
             "c024d69012485ed05d7eaa9617a96b81"),
        "mono_no_pt_640x192":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/mono_no_pt_640x192.zip",
             "9c2f071e35027c895a4728358ffc913a"),
        "stereo_no_pt_640x192":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/stereo_no_pt_640x192.zip",
             "41ec2de112905f85541ac33a854742d1"),
        "mono+stereo_no_pt_640x192":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/mono%2Bstereo_no_pt_640x192.zip",
             "46c3b824f541d143a45c37df65fbab0a"),
        "mono_1024x320":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/mono_1024x320.zip",
             "0ab0766efdfeea89a0d9ea8ba90e1e63"),
        "stereo_1024x320":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/stereo_1024x320.zip",
             "afc2f2126d70cf3fdf26b550898b501a"),
        "mono+stereo_1024x320":
            ("https://storage.googleapis.com/niantic-lon-static/research/monodepth2/mono%2Bstereo_1024x320.zip",
             "cdc5fc9b23513c07d5b19235d9ef08f7"),
        }

    if not os.path.exists("./monodepth2/models"):
        os.makedirs("models")

    model_path = os.path.join("./monodepth2/models", model_name)

In [15]:
from monodepth2 import networks
from monodepth2 import layers

# Cargar el modelo
download_model_if_doesnt_exist(model_name)
encoder_path = os.path.join("./monodepth2/models", model_name, "encoder.pth")
depth_decoder_path = os.path.join("./monodepth2/models", model_name, "depth.pth")

# # Cargando componentes del modelo
# encoder = MonoDecoder(num_input_images=1)
# depth_decoder = DepthDecoder(num_ch_enc=encoder.num_ch_enc, scales=range(4))

# loaded_dict_enc = torch.load(encoder_path, map_location='cpu')
# filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
# encoder.load_state_dict(filtered_dict_enc)

# loaded_dict = torch.load(depth_decoder_path, map_location='cpu')
# depth_decoder.load_state_dict(loaded_dict)

# encoder.eval()
# depth_decoder.eval()

# # Cargar y procesar cada imagen
# for image_file in os.listdir(input_folder):
#     if image_file.endswith((".png", ".jpg", ".jpeg")):
#         image_path = os.path.join(input_folder, image_file)
#         input_image = cv2.imread(image_path)
#         input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
#         input_image = cv2.resize(input_image, (640, 192))
#         input_image = transforms.ToTensor()(input_image).unsqueeze(0)

#         with torch.no_grad():
#             features = encoder(input_image)
#             outputs = depth_decoder(features)

#         disp = outputs[("disp", 0)]
#         disp_resized = torch.nn.functional.interpolate(
#             disp, (input_image.shape[2], input_image.shape[3]), mode="bilinear", align_corners=False)

#         # Normalizar el mapa de profundidad para visualización
#         disp_resized_np = disp_resized.squeeze().cpu().numpy()
#         vmax = np.percentile(disp_resized_np, 95)
#         normal_depth = np.clip(disp_resized_np / vmax, 0, 1)
#         depth_colormap = (normal_depth * 255).astype(np.uint8)
#         depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_MAGMA)

#         output_path = os.path.join(output_folder, image_file)
#         cv2.imwrite(output_path, depth_colormap)

# print("Procesamiento completado. Mapas de profundidad guardados en:", output_folder)

ModuleNotFoundError: No module named 'layers'

In [54]:
import glob

if os.path.isdir(input_folder):
    # Searching folder for images
    paths = glob.glob(os.path.join(input_folder, '*.{}'.format('jpg')))
    output_directory = input_folder
else:
    raise Exception("Can not find args.image_path: {}".format(input_folder))

print("-> Predicting on {:d} test images".format(len(paths)))

-> Predicting on 41 test images


In [59]:
from monodepth2.layers import disp_to_depth
from monodepth2.evaluate_depth import STEREO_SCALE_FACTOR
# PREDICTING ON EACH IMAGE IN TURN
with torch.no_grad():
    for idx, image_path in enumerate(paths):

        if image_path.endswith("_disp.jpg"):
            # don't try to predict disparity for a disparity image!
            continue

        # Load image and preprocess
        input_image = Image.open(image_path).convert('RGB')
        original_width, original_height = input_image.size
        input_image = input_image.resize((feed_width, feed_height), Image.LANCZOS)
        input_image = transforms.ToTensor()(input_image).unsqueeze(0)

        # PREDICTION
        input_image = input_image.to(device)
        features = encoder(input_image)
        outputs = depth_decoder(features)

        disp = outputs[("disp", 0)]
        disp_resized = torch.nn.functional.interpolate(
            disp, (original_height, original_width), mode="bilinear", align_corners=False)

        # Saving numpy file
        output_name = os.path.splitext(os.path.basename(image_path))[0]
        scaled_disp, depth = disp_to_depth(disp, 0.1, 100)

        name_dest_npy = os.path.join(output_folder, "{}_depth.npy".format(output_name))
        metric_depth = STEREO_SCALE_FACTOR * depth.device().numpy()
        np.save(name_dest_npy, metric_depth)

        # Saving colormapped depth image
        disp_resized_np = disp_resized.squeeze().cpu().numpy()
        vmax = np.percentile(disp_resized_np, 95)
        normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
        mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
        colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
        im = Image.fromarray(colormapped_im)

        name_dest_im = os.path.join(output_directory, "{}_disp.jpeg".format(output_name))
        im.save(name_dest_im)

        print("   Processed {:d} of {:d} images - saved predictions to:".format(
            idx + 1, len(paths)))
        print("   - {}".format(name_dest_im))
        print("   - {}".format(name_dest_npy))

print('-> Done!')

ModuleNotFoundError: No module named 'layers'

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image
import networks

# Configuración de la carpeta de imágenes y resultados
input_folder = './monocular_photos'
output_folder = './depth_maps'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Cargar el modelo
model_path = 'models/mono+stereo_640x192'
encoder_path = os.path.join(model_path, 'encoder.pth')
depth_decoder_path = os.path.join(model_path, 'depth.pth')

encoder = networks.ResnetEncoder(18, False)
depth_decoder = networks.DepthDecoder(num_ch_enc=encoder.num_ch_enc, scales=range(4))

loaded_dict_enc = torch.load(encoder_path, map_location='cpu')
filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
encoder.load_state_dict(filtered_dict_enc)

depth_decoder.load_state_dict(torch.load(depth_decoder_path, map_location='cpu'))

encoder.eval()
depth_decoder.eval()

# Procesar imágenes
with torch.no_grad():
    for image_file in os.listdir(input_folder):
        if image_file.endswith((".png", ".jpg", ".jpeg")):
            input_image = Image.open(os.path.join(input_folder, image_file)).convert('RGB')
            original_width, original_height = input_image.size
            input_image = input_image.resize((640, 192), Image.LANCZOS)
            input_image = transforms.ToTensor()(input_image).unsqueeze(0)

            # Predicción de profundidad
            features = encoder(input_image)
            outputs = depth_decoder(features)

            disp = outputs[("disp", 0)]
            disp_resized = torch.nn.functional.interpolate(
                disp, (original_height, original_width), mode="bilinear", align_corners=False)
            disp_resized_np = disp_resized.squeeze().cpu().numpy()

            # Guardar el resultado
            depth_filename = os.path.join(output_folder, os.path.splitext(image_file)[0] + '_depth.png')
            Image.fromarray((disp_resized_np * 255).astype(np.uint8)).save(depth_filename)

print("Procesamiento completado.")
